# 问题

In [30]:
link = "https://gkcx.eol.cn/soudaxue/queryProvinceScoreNum.html?schoolflag=&page=1&schoolyear=&argyear="

这个链接返回的页面是动态页面，其 HTML 文件本身只有空的表格和 JavaScript 代码，没有数据。当浏览器渲染页面时会执行 JavaScript 代码，此时 JavaScript 代码从网站服务器获取数据，然后展现在页面上。Python 并不会执行 JavaScript，所以 pandas 的 read_html 函数看到的是没有数据的页面。

In [ ]:
import pandas as pd
pd.read_html(link)

# 抓取动态页面

抓取动态页面的数据就是要找到网站的API,一般来说这些API都是返回JSON文件。比如说，我在中国教育在线网站查询北京大学的时候，有一个链接返回的是JSON 资源。

![image.png](query.png)

这个链接返回的JSON文件就是我们想要的数据：

![json.png](json.png)

分析这个API的参数

```
https://data-gkcx.eol.cn/soudaxue/queryProvinceScore.html?messtype=jsonp&lunum=1&callback=jQuery18306243269813396837_1527828613487&provinceforschool=&schooltype=&page=1&size=10&keyWord=%E5%8C%97%E4%BA%AC%E5%A4%A7%E5%AD%A6&schoolproperty=&schoolflag=&province=%E5%8C%97%E4%BA%AC&fstype=%E7%90%86%E7%A7%91&zhaoshengpici=&fsyear=&_=1527828616015
```

发现参数 messtype 的值是 jsonp，意思就是“请求的消息类型是 jsonp”，jsonp 就是 json 但是多了些我们不需要的东西，Python 不好直接处理，我们可以把 messtype 改成 json, 这个API就能直接返回 json 了。参数 page 和 size 关于分页的，网站默认是 `size=10` `page=1`，大概就是每页10行，请求第一页的数据。

In [ ]:
# pip install requests, or conda install requests
import requests

url = r"https://data-gkcx.eol.cn/soudaxue/queryProvinceScore.html?messtype=json"
# 每页10行，请求第2页
r = requests.get(url, data = {"page": 2, "size": 10})
r.json()

Json 数据很好操作

In [ ]:
row = "{schoolname}, {localprovince}, {province}, {studenttype}, {year}, {batch}, {var}, {var_score}"

for s in r.json()["school"]:
    print(row.format(**s))